In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
import cv2
import os,glob
import re
import sys
sys.path.append('/home/shuonan.chen/scratch_shuonan/code/pons_merfish_pipeline/processing/func')
from registerCCF_util import *
from prepare_img_utils import *
from utils import get_paths

def get_record(s):
    return {
        'filename':s.get('filename'),
        'height': s.get('height'),
        'width': s.get('width'), 
        'ox': s.get('anchoring')[0], 
        'oy': s.get('anchoring')[1],
        'oz': s.get('anchoring')[2],
        'ux': s.get('anchoring')[3],
        'uy': s.get('anchoring')[4],
        'uz': s.get('anchoring')[5], 
        'vx': s.get('anchoring')[6], 
        'vy': s.get('anchoring')[7],
        'vz': s.get('anchoring')[8],
    }

HOMEDIR = '/allen/aind/scratch/shuonan.chen/code/pons_merfish_pipeline/'
paths = get_paths()
print(paths.keys())

In [ ]:
scaling_factor=1/32;
all_file_path = '/allen/aind/scratch/shuonan.chen/code/pons_merfish_pipeline/processing/image_xml/*.jpg'
for f in glob.glob(all_file_path):
    print(os.path.basename(f).split('cell_img_')[1].split('.')[0])

In [ ]:
slicename_all = [os.path.basename(f).split('cell_img_')[1].split('.')[0] for f in glob.glob(all_file_path)]
slicename_all.sort()
print(slicename_all, len(slicename_all))

# batch process

In [ ]:
save_flag = True
plotting_2d = False
plotting_3d = False
slicenum = ['1']
slicenum=[int(i) for i in slicenum]

for name_of_slice in slicename_all:
    image_files=glob.glob(HOMEDIR + f'/processing/image_xml/*{name_of_slice}.jpg')
    # load your file
    flat_name=[]  
    for i in image_files:
        result=re.search('(.*).jpg', i)
        n=result.group(1)+('_nl.flat')
        flat_name+=[n]
    assert(len(image_files)==1)
    im = cv2.imread(image_files[0])
    h,w=im.shape[:2]
    json_name=image_files 
    d = {'slicenum': slicenum, 'json_name': json_name,'flat_name':flat_name}
    name_df=pd.DataFrame(data=d)
    neurons=pd.read_csv(HOMEDIR+f'/processing/filt_neurons_all/filt_neurons_{name_of_slice}.csv')
    neurons=neurons[neurons['slice'].isin(slicenum)]
    Max=neurons['x'].max()
    Min=neurons['y'].max()
    neurons['rescale_x']=neurons['x']*scaling_factor
    neurons['rescale_y']=neurons['y']*scaling_factor
    neurons['rescale_x'] = neurons['rescale_x'].astype(float)+500 # this was accounting for the padding that we created for the image
    neurons['rescale_y'] = neurons['rescale_y'].astype(float)+500
    
    # load visualign results 
    with open(HOMEDIR+f'/processing/visualign_rez/{name_of_slice}.json') as f:
        vafile=json.load(f)
    details={s.get('filename'):s for s in vafile['slices']}
    assert(name_of_slice in list(details.keys())[0])
    
    rez = neurons.groupby('slice', group_keys=False).apply(lambda g: get_adjusted_points(g.assign(slice=g.name), name_df,details)).reset_index(drop=True)
    neurons[['adjusted_x', 'adjusted_y']]= rez
    neurons_nl=neurons.copy()
    neurons_nl['clustid'] =neurons_nl['clustid'].astype(str)
    
    # load quickniii results 
    f = open(HOMEDIR+f'/processing/quicknii_rez/quicknii_{name_of_slice}.json','r')
    data=json.loads(f.read())
    anchor=pd.DataFrame.from_records([get_record(s) for s in data['slices']])
    f.close()

    vox_dfs = []
    for slice_num, df in neurons_nl.groupby('slice'):
        quicknii_cord = get_quicknii_cord(slice_num, df,name_df,anchor,h,w) # Height and Width of image file
        vox_cord = get_vox_cord(quicknii_cord)
        vox_dfs += [vox_cord]    
    vox_df = pd.concat(vox_dfs)
    
    if save_flag:
        vox_df.to_csv(HOMEDIR+f'/processing/registered_foo/registered_{name_of_slice}.csv',index=False)
    
    # optional plotting!
    if plotting_3d:
        plot_df = vox_df.copy()
        fig = px.scatter_3d(plot_df, x='y_CCF', y='z_CCF', z='x_CCF', color='clustid')
        fig.update_traces(marker=dict(size=.5),selector=dict(mode='markers'))
        fig.update_scenes(aspectmode='data')
        fig.show()
    elif plotting_2d:
        fig = px.scatter(plot_df, x='z_CCF', y='y_CCF', color='clustid')
        fig.update_traces(marker=dict(size=1),selector=dict(mode='markers'))
        fig.update_yaxes(scaleanchor='x', scaleratio=1)
        fig.show()
        

In [ ]:
flat_name=[]  
for i in image_files:
    result=re.search('(.*).jpg', i)
    n=result.group(1)+('_nl.flat')
    flat_name+=[n]
slicenum=[int(i) for i in slicenum]


In [ ]:
im = cv2.imread(image_files[0])
w=im.shape[1]
h=im.shape[0]

json_name=image_files 
d = {'slicenum': slicenum, 'json_name': json_name,'flat_name':flat_name}
name_df=pd.DataFrame(data=d)


In [ ]:
neurons=pd.read_csv(HOMEDIR+f'/processing/filt_neurons_all/filt_neurons_{name_of_slice}.csv')
neurons=neurons[neurons['slice'].isin(slicenum)]
Max=neurons['x'].max()
Min=neurons['y'].max()
neurons['rescale_x']=(neurons['x']*scaling_factor)
neurons['rescale_y']=(neurons['y']*scaling_factor)


# invert warping process

In [ ]:

with open("visualign.json") as f:
    vafile=json.load(f)
details={s.get('filename'):s for s in vafile['slices']}
assert(name_of_slice in list(details.keys())[0])

neurons[['adjusted_x', 'adjusted_y']] = neurons.groupby('slice').apply(get_adjusted_points).reset_index(drop=True)
neurons_nl=neurons.copy()
neurons_nl['clustid'] =neurons_nl['clustid'].astype(str)

# invert affine transform 

In [ ]:
def get_record(s):
    return {
        'filename':s.get('filename'),
        'height': s.get('height'),
        'width': s.get('width'), 
        'ox': s.get('anchoring')[0], 
        'oy': s.get('anchoring')[1],
        'oz': s.get('anchoring')[2],
        'ux': s.get('anchoring')[3],
        'uy': s.get('anchoring')[4],
        'uz': s.get('anchoring')[5], 
        'vx': s.get('anchoring')[6], 
        'vy': s.get('anchoring')[7],
        'vz': s.get('anchoring')[8],
    }

In [ ]:
f = open('quicknii.json','r')
data=json.loads(f.read())
anchor=pd.DataFrame.from_records([get_record(s) for s in data['slices']])
f.close()

vox_dfs = []
for slice_num, df in neurons_nl.groupby('slice'):
    quicknii_cord = get_quicknii_cord(slice_num, df,h,w) # Height and Width of image file
    vox_cord = get_vox_cord(quicknii_cord)
    vox_dfs += [vox_cord]    
vox_df = pd.concat(vox_dfs)


# plot the results 

In [ ]:

plot_df = vox_df.copy()
fig = px.scatter_3d(plot_df, x='y_CCF', y='z_CCF', z='x_CCF', color='clustid')
fig.update_traces(marker=dict(size=.5),
                  selector=dict(mode='markers'))
fig.update_scenes(aspectmode='data')
fig.show()